In [1]:
# importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import the dataset 

df = pd.read_csv('telecom_customer_churn.csv')
df.head()

# printing the columns of the dataset
#print(df.columns)

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [3]:
# removing spaces from the column names
df.columns = df.columns.str.replace(' ', '_')

# printing the columns of the dataset after removing spaces
print(df.columns)

Index(['Customer_ID', 'Gender', 'Age', 'Married', 'Number_of_Dependents',
       'City', 'Zip_Code', 'Latitude', 'Longitude', 'Number_of_Referrals',
       'Tenure_in_Months', 'Offer', 'Phone_Service',
       'Avg_Monthly_Long_Distance_Charges', 'Multiple_Lines',
       'Internet_Service', 'Internet_Type', 'Avg_Monthly_GB_Download',
       'Online_Security', 'Online_Backup', 'Device_Protection_Plan',
       'Premium_Tech_Support', 'Streaming_TV', 'Streaming_Movies',
       'Streaming_Music', 'Unlimited_Data', 'Contract', 'Paperless_Billing',
       'Payment_Method', 'Monthly_Charge', 'Total_Charges', 'Total_Refunds',
       'Total_Extra_Data_Charges', 'Total_Long_Distance_Charges',
       'Total_Revenue', 'Customer_Status', 'Churn_Category', 'Churn_Reason'],
      dtype='object')


In [4]:
# data check 

df[df['Phone_Service'] == 'No']['Avg_Monthly_Long_Distance_Charges'].value_counts()

Series([], Name: count, dtype: int64)

In [5]:
# missing value imputation 

# part 1 : All internet related columns 

internet_binary_features = [
    'Online_Security', 'Online_Backup', 'Device_Protection_Plan',
    'Premium_Tech_Support', 'Streaming_TV', 'Streaming_Movies',
    'Streaming_Music', 'Unlimited_Data' ]

# Step 1 : Convert the columns with 'No' and 'Yes' values to 0 and 1 respectively

df[internet_binary_features] = df[internet_binary_features].replace({'Yes': 1, 'No': 0})

# Step 2 : Convert the missing values of internet binary features to 0 if Internet Service = No ( We will overwrite the values later)

df.loc[df['Internet_Service'] == 'No', internet_binary_features] = 0

# Step 3 : Create summary feature for internet service (Add a default +1 for every one who has internet to differentiate for 0 due no internet or no internet features)

df['Num_Internet_Features'] = df[internet_binary_features].sum(axis=1) + 1


# missing values handled here 
df.loc[df['Internet_Service'] == 'No', 'Num_Internet_Features'] = 0

# Now handle internet type column ( We will one hot encode this column later)

df.loc[df['Internet_Service'] == 'No', 'Internet_Type'] = 'None'

# Now handle the avg Avg_Monthly_GB_Download

df.loc[df['Internet_Service'] == 'No', 'Avg_Monthly_GB_Download'] = 0

# Now we do not need "Internet service" column to avoid multicollinearlity and individual internet columns, so we can drop it

df.drop('Internet_Service', axis=1, inplace=True)

df = df.drop(columns=internet_binary_features)

# part 2 : All columns related to phone service 

df.loc[df['Phone_Service'] == 'No', 'Avg_Monthly_Long_Distance_Charges'] = 0

# Step 1: Replace 'Yes'/'No' with 1/0
df['Multiple_Lines'] = df['Multiple_Lines'].replace({'Yes': 1, 'No': 0})

# Step 2: Set Multiple_Lines = 0 where Phone_Service == 'No' (We will overwrite the values later)
df.loc[df['Phone_Service'] == 'No', 'Multiple_Lines'] = 0

# Step 3: Create a new feature (Add a deafult +1 , to avoid 0 for No and missing both)
df['Has_Multiple_Lines'] = df['Multiple_Lines'] + 1 

# Step 4: Set Has_Multiple_Lines = -1 for users with no phone service
df.loc[df['Phone_Service'] == 'No', 'Has_Multiple_Lines'] = 0

# Step 5: Drop the Multiple_Lines column and phone service column
df.drop('Multiple_Lines', axis=1, inplace=True)
df.drop('Phone_Service', axis=1, inplace=True)

# part 3 : Blank offer means no offer
df.loc[df['Offer'].isna() , ['Offer']] = 'No Offer'

C:\Users\abhin\AppData\Local\Temp\ipykernel_22336\1602245396.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[internet_binary_features] = df[internet_binary_features].replace({'Yes': 1, 'No': 0})
C:\Users\abhin\AppData\Local\Temp\ipykernel_22336\1602245396.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Multiple_Lines'] = df['Multiple_Lines'].replace({'Yes': 1, 'No': 0})


In [6]:
# Null value check 
df.isnull().sum()[(df.isnull().sum() > 0).values]

Churn_Category    5174
Churn_Reason      5174
dtype: int64

In [7]:
df['Num_Internet_Features'].value_counts()

Num_Internet_Features
0.0    1526
5.0     880
4.0     857
3.0     829
6.0     826
2.0     693
7.0     668
8.0     458
9.0     225
1.0      81
Name: count, dtype: int64

In [8]:
# datatype check
df.dtypes

# the churn column need to be converted to 0,1

condition = [df['Customer_Status'].isin(['Stayed','Joined']),df['Customer_Status'].isin(['Churned'])]

df['Churn_status'] = np.select(condition,[0,1],default=0)


In [9]:
# data check
df.groupby(['Churn_status','Customer_Status']).agg({'Customer_ID': 'count'}).reset_index()
#df.groupby('Churn_status')['Customer_Status'].agg({'Customer_id' : 'count'}).reset_index()

# data columns check 
df.dtypes

Customer_ID                           object
Gender                                object
Age                                    int64
Married                               object
Number_of_Dependents                   int64
City                                  object
Zip_Code                               int64
Latitude                             float64
Longitude                            float64
Number_of_Referrals                    int64
Tenure_in_Months                       int64
Offer                                 object
Avg_Monthly_Long_Distance_Charges    float64
Internet_Type                         object
Avg_Monthly_GB_Download              float64
Contract                              object
Paperless_Billing                     object
Payment_Method                        object
Monthly_Charge                       float64
Total_Charges                        float64
Total_Refunds                        float64
Total_Extra_Data_Charges               int64
Total_Long

In [10]:
# dump the data to csv
df.to_csv('telecom_customer_churn_cleaned.csv', index=False)